In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re
import copy

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='3'

to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']

In [3]:
# Read in fmri events
fn = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv' %(subnum, subnum, runnum))
events = pd.read_csv(fn, sep='\t')

# Read in behavioral data with modeled value and RPE estimates
behavior = pd.read_csv(behavior_path)

# Extract the correct subnum and runnum from behavioral data
run_behavior = behavior.query('subnum == %d & session == %d'%(int(subnum), int(runnum)))

# Demean columns that might be used for parametric regressors
# demean_cols = ['probFractalDraw', 'leftBundleVal', 'rightBundleVal', 'leftLotteryEV', 'rightLotteryEV', 'leftQValue', 'rightQValue', 'reward', 'ppe', 'leftQVAdv', 'leftEVAdv', 'leftbundleValAdv', 'leftFractalRpe', 'rightFractalRpe', 'valChosen', 'valUnchosen', 'valChosenLottery', 'valUnchosenLottery', 'valChosenFractal', 'valUnchosenFractal', 'leftFractalReward', 'rightFractalReward']
# demean_df = run_behavior[demean_cols]
# demean_df = demean_df - demean_df.mean()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [8]:
rewardEvents = events.query('trial_type == "reward"').reset_index(drop=True)
rewardedTrials = run_behavior.reset_index(drop=True).query("reward > 0").index
rewardedEvents = rewardEvents.iloc[rewardedTrials,:]
cond_rewarded_par = rewardedEvents[['onset', 'duration']].reset_index(drop=True)
cond_rewarded_par['trial_type'] = 'rewarded_par'
rewards = np.array(run_behavior.query("reward > 0")['reward'])
cond_rewarded_par['modulation'] = rewards - rewards.mean()


array([1. , 0.7, 0.9, 0.7, 1. , 1. , 2.5, 1. , 1. , 1. , 1. , 1. , 2.5,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 1. , 0.9, 1. , 1. ])

In [9]:
rewards

array([1. , 0.7, 0.9, 0.7, 1. , 1. , 2.5, 1. , 1. , 1. , 1. , 1. , 2.5,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 2. , 1. , 0.9, 1. , 1. ])

In [10]:
rewards - rewards.mean()

array([-0.12307692, -0.42307692, -0.22307692, -0.42307692, -0.12307692,
       -0.12307692,  1.37692308, -0.12307692, -0.12307692, -0.12307692,
       -0.12307692, -0.12307692,  1.37692308, -0.12307692, -0.12307692,
       -0.12307692, -0.12307692, -0.12307692, -0.12307692, -0.12307692,
       -0.12307692,  0.87692308, -0.12307692, -0.22307692, -0.12307692,
       -0.12307692])

In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt))
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)